In [4]:
from typing import Any, List, Optional

from langchain_core.documents import Document
from rdflib import Dataset
from rdflib import Dataset
from rdflib.namespace import RDF
import os 
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

load_dotenv()



False

In [5]:
# g = Dataset()
# g.parse("data/connected-to.trig", "format=trig")

# for count, (ns, np, o, nz) in enumerate(g.quads((None, None, None, None))): 
#     if count < 1000:        
#         s =ns.split('#')[1] if '#' in ns else ns
#         p =np.split('#')[1] if '#' in np else np
#         z =nz.split('#')[1] if '#' in nz else nz
        
#         property = None

#         #if(p.endswith("-uid")): property = 'uid'
#         #if(p.endswith("-code")): property = 'code'
#         if(p.endswith("-displayLabel")): property = 'label'

#         if(property != None): 
#             split = s.split("-")
#             type = split[0]
#             code = split[1]

#             typeLabel = None

#             if(type.endswith("ChannelReach")): typeLabel = 'channel reach'
#             if(type.endswith("Project")): typeLabel = 'project'
#             if(type.endswith("Dam")): typeLabel = 'dam'

#             if(len(o) > 0): 
#                     docs.append(Document(
#                         page_content=f"The location {o} has the code {code}."
#                     ))        
# print(docs)


In [6]:
# from typing import List

# from langchain_core.documents import Document

# docs: List[Document] = []

# docs.append(Document(page_content=f"The school South Park High School is within school district Park Couny School District"))
# docs.append(Document(page_content=f"The school Guffey Charter School is within school district Park Couny School District"))
# docs.append(Document(page_content=f"The school Lake George Charter School is within school district Park Couny School District"))
# docs.append(Document(page_content=f"The school South Park Middle School is within school district Park Couny School District"))
# docs.append(Document(page_content=f"The school Edith Teter Elementary School is within school district Park Couny School District"))

# docs.append(Document(page_content=f"The school Lawrence Elementary School is within school district Jefferson Couny School District"))
# docs.append(Document(page_content=f"The school Falcon Bluffs Middle School is within school district Jefferson Couny School District"))
# docs.append(Document(page_content=f"The school Summit Ridge Middle School is within school district Jefferson Couny School District"))
# docs.append(Document(page_content=f"The school Kyffin Elementary School is within school district Jefferson Couny School District"))
# docs.append(Document(page_content=f"The school Mount View Youth Services Center is within school district Jefferson Couny School District"))
# docs.append(Document(page_content=f"The school South Lakewood Elementary School is within school district Jefferson Couny School District"))

# docs.append(Document(page_content=f"The school Flagstone Elementary School is within school district Douglas Couny School District"))
# docs.append(Document(page_content=f"The school Highlands Ranch High School is within school district Douglas Couny School District"))
# docs.append(Document(page_content=f"The school Fox Creek Elementary School is within school district Douglas Couny School District"))
# docs.append(Document(page_content=f"The school Parker Performing Arts is within school district Douglas Couny School District"))
# docs.append(Document(page_content=f"The school Leman Classical Academy is within school district Douglas Couny School District"))
# docs.append(Document(page_content=f"The school Stem School Highlands Ranch is within school district Douglas Couny School District"))

# print(docs)


In [7]:
docs: List[Document] = []

dataset = Dataset()
dataset.parse("data/lpg-schools/schools.trig", "format=trig")

outer_query = """
PREFIX lpgvs: <https://localhost:4200/lpg/SchoolZone/1/rdfs#>
PREFIX lpgs: <https://localhost:4200/lpg/rdfs#>

SELECT ?zoneCode ?zoneLabel
WHERE {
    ?zone a lpgvs:SchoolZone .
    ?zone lpgs:GeoObjectType-code ?zoneCode .
    ?zone lpgs:GeoObjectType-displayLabel ?zoneLabel .
}
LIMIT 10"""

for c in dataset.graphs():  # doctest: +SKIP
    qres = c.query(outer_query)
    for row in qres:
        page_content = ""

        inner_query = (
        f"PREFIX lpgvs: <https://localhost:4200/lpg/SchoolZone/1/rdfs#>\n"
        f"PREFIX lpgs: <https://localhost:4200/lpg/rdfs#>\n"
        
        f"SELECT ?schoolCode ?schoolLabel\n"
        f"WHERE {{\n"
        f"    ?parent lpgvs:HasSchoolZone lpgv:SchoolZone-{row.zoneCode} .\n"
        f"    ?parent lpgs:GeoObjectType-code ?schoolCode .\n"
        f"    ?parent lpgs:GeoObjectType-displayLabel ?schoolLabel .\n"
        f"}}\n"
        f"LIMIT 100\n")

        ires = c.query(inner_query)
        for result in ires:
            page_content += f"The school {result.schoolLabel} is within the school district {row.zoneLabel}. "

        docs.append(Document(
            page_content=page_content,
            metadata={'zone':f"{row.zoneLabel}", 'code': f"{row.zoneCode}", 'source': "rdf"}
        ))        


print(docs)


[Document(page_content='The school NORTHSIDE ELEMENTARY is within the school district Black Oak Mine Unified School District. The school GEORGETOWN ELEMENTARY is within the school district Black Oak Mine Unified School District. The school AMERICAN RIVER CHARTER is within the school district Black Oak Mine Unified School District. The school GOLDEN SIERRA JUNIOR SENIOR HIGH is within the school district Black Oak Mine Unified School District. The school OTTER CREEK ELEMENTARY is within the school district Black Oak Mine Unified School District. The school DIVIDE HIGH is within the school district Black Oak Mine Unified School District. ', metadata={'zone': 'Black Oak Mine Unified School District', 'code': '0605240'}), Document(page_content='The school DATELAND ELEMENTARY SCHOOL is within the school district Hyder Elementary District. ', metadata={'zone': 'Hyder Elementary District', 'code': '0403900'}), Document(page_content='The school BELLAIRE ELEMENTARY SCHOOL is within the school d

In [8]:
MODEL = 'llama2'
#MODEL = 'mixtral'

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()


In [ ]:
from langchain_elasticsearch import ElasticsearchStore
from uuid import uuid4

vectorstore = ElasticsearchStore(
    es_url="http://localhost:9200",
    index_name="langchain_index",
    embedding=embeddings,
    es_user="elastic",
    es_password="elastic",
)

uuids = [str(uuid4()) for _ in range(len(docs))]

vectorstore.add_documents(documents=docs, ids=uuids)

In [12]:
# from langchain_community.vectorstores import DocArrayInMemorySearch

# vectorstore = DocArrayInMemorySearch.from_documents(docs, embedding=embeddings)

In [9]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke('Tell me a joke')

"Sure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope you found that amusing! Do you want to hear another one?"

In [10]:
from langchain.prompts import PromptTemplate

template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:
"""

prompt = PromptTemplate.from_template(template)

print(prompt.format(context="Here is some context", question="Here is a question"))

chain = prompt | model | parser


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: Here is a question 

Context: Here is some context 

Answer:



In [11]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [13]:
from operator import itemgetter

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.2}
)

chain = (
    {
        "context": itemgetter('question') | retriever,
        "question": itemgetter('question')
    } | 
prompt | model | parser)


In [14]:
retriever.invoke('GEORGETOWN ELEMENTARY')

[Document(page_content='The school HOBBS WILLIAMS EL is within the school district Grand Prairie Independent School District. The school CROCKETT EARLY EDUCATION SCHOOL is within the school district Grand Prairie Independent School District. The school ADVANTAGE ACADEMY is within the school district Grand Prairie Independent School District. The school DIGITAL ARTS & TECHNOLOGY ACADEMY AT ADAMS MIDDLE is within the school district Grand Prairie Independent School District. The school SOUTH GRAND PRAIRIE H S is within the school district Grand Prairie Independent School District. The school RONALD W REAGAN MIDDLE is within the school district Grand Prairie Independent School District. The school SAM RAYBURN EL STEAM ACADEMY is within the school district Grand Prairie Independent School District. The school ERVIN C WHITT EL is within the school district Grand Prairie Independent School District. The school JOHN A DUBISKI CAREER H S is within the school district Grand Prairie Independent 

In [15]:
chain.invoke({
    "question": "What is the school district of GEORGETOWN ELEMENTARY?",
})

'The school district of GEORGETOWN ELEMENTARY is GRAND PRAIRIE INDEPENDENT SCHOOL DISTRICT.'

In [16]:
questions = [
    "What is the school district of GEORGETOWN ELEMENTARY school?",
    "What is the school district of Highlands Ranch High School?",
    "What is the school district of COMMUNITY PARTNERSHIP CHARTER SCHOOL?",
    "What schools are part of the Black Oak Mine Unified School District?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question })}")
    print()

Question: What is the school district of GEORGETOWN ELEMENTARY school?
Answer: I don't know the school district of GEORGETOWN ELEMENTARY school. The context you provided does not contain enough information to determine the school district. Please provide more context or clarify which question you are asking so I can better assist you.

Question: What is the school district of Highlands Ranch High School?
Answer: Sure! Here are the schools mentioned in the text, along with their locations and districts:

1. Environmental Science Academy - within Grand Prairie Independent School District
2. Andrew Jackson Middle - within Grand Prairie Independent School District
3. Mike Mosley El - within Grand Prairie Independent School District
4. David Daniels El Academy of Science & Math - within Grand Prairie Independent School District
5. Harry S Truman Middle - within Grand Prairie Independent School District
6. Global Leadership Academy at Barbara Bush El - within Grand Prairie Independent School